In [2]:
# install KoNLPy
# https://mebadong.tistory.com/95
# https://konlpy.org/ko/latest/install/

## intall java
# ! sudo apt update
# ! sudo apt upgrade
# ! sudo apt install openjdk-17-jdk
## check java
# ! java -version
# ! javac -version
## set environment
# ! echo "# >>> set JAVA environment" >> ~/.zshrc
################# ! echo "export JAVA_HOME=$(dirname $(dirname $(readlink -f $(which java))))" >> ~/.zshrc
# ! echo "export PATH=$PATH:$JAVA_HOME/bin" >> ~/.zshrc
# ! echo "# <<< set JAVA environment" >> ~/.zshrc
# ! source ~/.zshrc
## check environment
# ! echo $JAVA_HOME
## Install dependencies
# ! sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
## Install KoNLPy
# ! pip install --upgrade pip
# ! pip install konlpy
## Install MeCab
# ! sudo apt-get -y install automake
# ! zsh <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


In [3]:
# Install gensim
# ! pip install gensim


In [1]:
import os
import re
from tqdm import tqdm
from konlpy.tag import Mecab
from gensim.models import Word2Vec


### downlad data


In [5]:
# wiki dump data download
# ! wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2


In [6]:
# install wikiextractor and parse dump data
# ! pip install wikiextractor
# ! python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2


### check data


In [7]:
%ls

gensim-kowiki.ipynb                   kowiki.txt  w2v-models/
kowiki-latest-pages-articles.xml.bz2  text/


In [8]:
# check text dir
print(os.listdir("text"))
print(os.listdir("text/AA"))


['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ']
['wiki_00', 'wiki_01', 'wiki_02', 'wiki_03', 'wiki_04', 'wiki_05', 'wiki_06', 'wiki_07', 'wiki_08', 'wiki_09', 'wiki_10', 'wiki_11', 'wiki_12', 'wiki_13', 'wiki_14', 'wiki_15', 'wiki_16', 'wiki_17', 'wiki_18', 'wiki_19', 'wiki_20', 'wiki_21', 'wiki_22', 'wiki_23', 'wiki_24', 'wiki_25', 'wiki_26', 'wiki_27', 'wiki_28', 'wiki_29', 'wiki_30', 'wiki_31', 'wiki_32', 'wiki_33', 'wiki_34', 'wiki_35', 'wiki_36', 'wiki_37', 'wiki_38', 'wiki_39', 'wiki_40', 'wiki_41', 'wiki_42', 'wiki_43', 'wiki_44', 'wiki_45', 'wiki_46', 'wiki_47', 'wiki_48', 'wiki_49', 'wiki_50', 'wiki_51', 'wiki_52', 'wiki_53', 'wiki_54', 'wiki_55', 'wiki_56', 'wiki_57', 'wiki_58', 'wiki_59', 'wiki_60', 'wiki_61', 'wiki_62', 'wiki_63', 'wiki_64', 'wiki_65', 'wiki_66', 'wiki_67', 'wiki_68', 'wiki_69', 'wiki_70', 'wiki_71', 'wiki_72', 'wiki_73', 'wiki_74', 'wiki_75', 'wiki_76', 'wiki_77', 'wiki_78', 'wiki_79', 'wiki_80', 'wiki_81', 'wiki_82', 'wiki_83', 'wiki_84', 'wi

In [9]:
with open("text/AA/wiki_99", "r") as f:
    lines = f.readlines()
    for line in lines[:5]:
        print(line)
    print("...")
    for line in lines[-5:]:
        print(line)


<doc id="77601" url="https://ko.wikipedia.org/wiki?curid=77601" title="소학언해">

소학언해



소학언해(小學諺解)는 중국의 주자가 지은 《소학》(小學)을 조선 선조 20년인 1587년에 정구 등 학자들이 한국어로 언해한 책이다. 15세기에 제작된 훈민정음 이후 변화된, 16세기 말 한국어의 모습을 잘 보여준다.

서.

...
대문자 O 표기법







</doc>



### parse data


In [10]:
# get all wifi file's path
def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)


filepaths = list_wiki("text")
print(len(filepaths))


912


In [11]:
%%time
# concat all wiki files to kowiki.txt
concated_file = "kowiki.txt"
with open(concated_file, "w") as outfile:
    for filename in tqdm(filepaths):
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

f = open(concated_file, "r")
lines = f.read().splitlines()


100%|██████████| 912/912 [00:12<00:00, 74.24it/s]


CPU times: user 9.06 s, sys: 7.67 s, total: 16.7 s
Wall time: 23.8 s


In [12]:
print(len(lines))
print(lines[:5])


10364934
['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">', '지미 카터', '', '제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.', '생애.']


In [13]:
%%time
# erase empty lines
non_empty_lines = []
for line in tqdm(lines):
    if line:
        non_empty_lines.append(line)
print(len(non_empty_lines))
for line in non_empty_lines[:5]:
    print(line)


100%|██████████| 10364934/10364934 [00:02<00:00, 4005630.09it/s]

7001189
<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">
지미 카터
제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.
생애.
어린 시절.
CPU times: user 2.17 s, sys: 422 ms, total: 2.59 s
Wall time: 2.59 s


### tokenization with mecab


In [14]:
%%time
# tokenization
mecab = Mecab()

result = []
for line in tqdm(non_empty_lines):
    result.append(mecab.morphs(line))
print(len(result))


100%|██████████| 7001189/7001189 [15:25<00:00, 7561.83it/s]  

7001189
CPU times: user 11min 31s, sys: 2min 47s, total: 14min 19s
Wall time: 15min 25s


In [15]:
print(result[:10])


[['<', 'doc', 'id', '="', '5', '"', 'url', '="', 'https', ':', '/', '/', 'ko', '.', 'wikipedia', '.', 'org', '/', 'wiki', '?', 'curid', '=', '5', '"', 'title', '="', '지미', '카터', '">'], ['지미', '카터'], ['제임스', '얼', '카터', '주니어', '(', ',', '1924', '년', '10', '월', '1', '일', '~', ')', '는', '민주당', '출신', '미국', '39', '대', '대통령', '(', '1977', '년', '~', '1981', '년', ')', '이', '다', '.'], ['생애', '.'], ['어린', '시절', '.'], ['지미', '카터', '는', '조지', '아주', '섬터', '카운티', '플', '레인스', '마을', '에서', '태어났', '다', '.'], ['조지아', '공과', '대학교', '를', '졸업', '하', '였', '다', '.', '그', '후', '해군', '에', '들어가', '전함', '·', '원자력', '·', '잠수함', '의', '승무원', '으로', '일', '하', '였', '다', '.', '1953', '년', '미국', '해군', '대위', '로', '예편', '하', '였', '고', '이후', '땅콩', '·', '면화', '등', '을', '가꿔', '많', '은', '돈', '을', '벌', '었', '다', '.', '그', '의', '별명', '이', '"', '땅콩', '농부', '"', '(', 'Peanut', 'Farmer', ')', '로', '알려졌', '다', '.'], ['정계', '입문', '.'], ['1962', '년', '조지', '아주', '상원', '의원', '선거', '에서', '낙선', '하', '나', '그', '선거', '가', '부정', '선거', '였음을', 

### train Word2Vec


- https://radimrehurek.com/gensim/models/word2vec.html?highlight=word2vec#module-gensim.models.word2vec


In [16]:
# %%time
# model_cbow_5_10 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=5,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=0,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=10,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_cbow_5_10.save("w2v-models/model_cbow_5_10.model")


CPU times: user 49min 22s, sys: 5min 7s, total: 54min 30s
Wall time: 19min 38s


In [17]:
# %%time
# model_cbow_15_10 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=15,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=0,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=10,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_cbow_15_10.save("w2v-models/model_cbow_15_10.model")


CPU times: user 54min 35s, sys: 5min 8s, total: 59min 44s
Wall time: 19min 54s


In [18]:
# %%time
# model_cbow_30_10 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=30,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=0,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=10,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_cbow_30_10.save("w2v-models/model_cbow_30_10.model")


CPU times: user 1h 4min 19s, sys: 5min 30s, total: 1h 9min 49s
Wall time: 21min 29s


In [19]:
# %%time
# model_sg_5_10 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=5,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=10,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_sg_5_10.save("w2v-models/model_sg_5_10.model")


CPU times: user 3h 46min 49s, sys: 6min 23s, total: 3h 53min 12s
Wall time: 33min 11s


In [20]:
# %%time
# model_sg_15_10 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=15,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=10,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_sg_15_10.save("w2v-models/model_sg_15_10.model")


CPU times: user 8h 58min 21s, sys: 6min 33s, total: 9h 4min 54s
Wall time: 1h 4min 50s


In [21]:
# %%time
# model_sg_30_10 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=30,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=10,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_sg_30_10.save("w2v-models/model_sg_30_10.model")


CPU times: user 15h 8min 50s, sys: 5min 50s, total: 15h 14min 41s
Wall time: 1h 41min 35s


In [16]:
# %%time
# model_sg_15_0 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=15,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=0,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_sg_15_0.save("w2v-models/model_sg_15_0.model")


CPU times: user 5min 14s, sys: 3min 1s, total: 8min 16s
Wall time: 10min 31s


In [17]:
# %%time
# model_sg_15_2 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=15,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=2,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_sg_15_2.save("w2v-models/model_sg_15_2.model")


CPU times: user 2h 10min 20s, sys: 4min 17s, total: 2h 14min 37s
Wall time: 19min 25s


In [18]:
# %%time
# model_sg_15_5 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=15,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=5,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_sg_15_5.save("w2v-models/model_sg_15_5.model")


CPU times: user 4h 34min, sys: 5min 22s, total: 4h 39min 23s
Wall time: 32min 24s


In [19]:
# %%time
# model_sg_15_20 = Word2Vec(
#     sentences=result,
#     vector_size=100,  # 100, Dimensionality of the word vectors.
#     window=15,  # Maximum distance between the current and predicted word within a sentence.
#     min_count=5,  # Ignores all words with total frequency lower than this.
#     workers=10,
#     sg=1,  # Training algorithm: 1 for skip-gram; otherwise CBOW.
#     negative=20,  # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.
#     epochs=5,  # 5, Number of iterations (epochs) over the corpus.
#     batch_words=10000,  # 10000, Target size (in words) for batches of examples passed to worker threads
#     compute_loss=True,  #  If True, computes and stores loss value which can be retrieved using get_latest_training_loss().
# )

# model_sg_15_20.save("w2v-models/model_sg_15_20.model")


CPU times: user 16h 16min 43s, sys: 4min 35s, total: 16h 21min 19s
Wall time: 1h 42min 2s


In [2]:
import os
import re
from tqdm import tqdm
from konlpy.tag import Mecab
from gensim.models import Word2Vec


In [3]:
models=[]
models.append(Word2Vec.load("w2v-models/model_cbow_5_10.model"))
models.append(Word2Vec.load("w2v-models/model_cbow_15_10.model"))
models.append(Word2Vec.load("w2v-models/model_cbow_30_10.model"))
models.append(Word2Vec.load("w2v-models/model_sg_5_10.model"))
models.append(Word2Vec.load("w2v-models/model_sg_15_10.model"))
models.append(Word2Vec.load("w2v-models/model_sg_30_10.model"))
models.append(Word2Vec.load("w2v-models/model_sg_15_0.model"))
models.append(Word2Vec.load("w2v-models/model_sg_15_2.model"))
models.append(Word2Vec.load("w2v-models/model_sg_15_5.model"))
models.append(Word2Vec.load("w2v-models/model_sg_15_20.model"))


In [26]:
for model in models:
    print(f"\n[+] sg: {model.sg}, window: {model.window}, negative: {model.negative}")
    for w in model.wv.most_similar(positive=["대단"], negative=[], topn=5):
        print(f"{w[0]:10}: {round(w[1], 3)}")
        


[+] sg: 0, window: 5, negative: 10
굉장        : 0.91
특출        : 0.811
어마어마      : 0.79
훌륭        : 0.782
암울        : 0.772

[+] sg: 0, window: 15, negative: 10
굉장        : 0.9
훌륭        : 0.832
어마어마      : 0.795
탁월        : 0.759
특출        : 0.744

[+] sg: 0, window: 30, negative: 10
굉장        : 0.878
훌륭        : 0.84
어마어마      : 0.781
탁월        : 0.743
막강        : 0.731

[+] sg: 1, window: 5, negative: 10
굉장        : 0.866
훌륭        : 0.779
특출        : 0.77
놀라운       : 0.729
놀랄        : 0.724

[+] sg: 1, window: 15, negative: 10
굉장        : 0.867
훌륭        : 0.773
놀랄        : 0.758
놀라운       : 0.743
감탄        : 0.737

[+] sg: 1, window: 30, negative: 10
굉장        : 0.869
훌륭        : 0.795
너무나       : 0.783
평했        : 0.768
극찬        : 0.763

[+] sg: 1, window: 15, negative: 0
러프버러      : 0.452
仁恕        : 0.429
lanceolatus: 0.421
콧         : 0.409
Sno       : 0.406

[+] sg: 1, window: 15, negative: 2
굉장        : 0.843
훌륭        : 0.778
인상깊       : 0.738
놀라운       : 0.737
감탄        :

In [25]:
for model in models:
    print(f"\n[+] sg: {model.sg}, window: {model.window}, negative: {model.negative}")
    for w in model.wv.most_similar(positive=["좋"], negative=[], topn=5):
        print(f"{w[0]:10}: {round(w[1], 3)}")
        


[+] sg: 0, window: 5, negative: 10
괜찮        : 0.777
높         : 0.761
많         : 0.756
낮         : 0.739
귀찮        : 0.709

[+] sg: 0, window: 15, negative: 10
괜찮        : 0.808
많         : 0.759
높         : 0.752
낮         : 0.718
귀찮        : 0.699

[+] sg: 0, window: 30, negative: 10
괜찮        : 0.817
많         : 0.748
높         : 0.739
낮         : 0.705
귀찮        : 0.62

[+] sg: 1, window: 5, negative: 10
괜찮        : 0.824
나쁘        : 0.767
녹록        : 0.746
윤건희       : 0.728
그닥        : 0.713

[+] sg: 1, window: 15, negative: 10
괜찮        : 0.826
잘         : 0.775
윤건희       : 0.719
들을수록      : 0.708
해졌으면      : 0.703

[+] sg: 1, window: 30, negative: 10
잘         : 0.798
보여        : 0.737
괜찮        : 0.737
생각        : 0.734
보         : 0.719

[+] sg: 1, window: 15, negative: 0
Kafka     : 0.461
언소        : 0.456
원생생물      : 0.45
千年        : 0.438
베른슈타인     : 0.423

[+] sg: 1, window: 15, negative: 2
잘         : 0.794
괜찮        : 0.787
보여        : 0.702
싶         : 0.693
못       

In [24]:
for model in models:
    print(f"\n[+] sg: {model.sg}, window: {model.window}, negative: {model.negative}")
    for w in model.wv.most_similar(positive=["못"], negative=[], topn=5):
        print(f"{w[0]:10}: {round(w[1], 3)}")
        


[+] sg: 0, window: 5, negative: 10
걱정        : 0.641
못하        : 0.632
피곤        : 0.616
괴롭        : 0.61
가려        : 0.593

[+] sg: 0, window: 15, negative: 10
못하        : 0.655
걱정        : 0.628
무서워       : 0.618
가려        : 0.615
막막        : 0.608

[+] sg: 0, window: 30, negative: 10
무서워       : 0.649
괴롭        : 0.635
걱정        : 0.629
못하        : 0.626
멀쩡        : 0.621

[+] sg: 1, window: 5, negative: 10
미더워       : 0.781
마땅        : 0.778
다칠까       : 0.775
뻔한데       : 0.762
괜한        : 0.757

[+] sg: 1, window: 15, negative: 10
그래도       : 0.779
괜찮        : 0.764
뻔한데       : 0.759
그렇게       : 0.74
질까봐       : 0.74

[+] sg: 1, window: 30, negative: 10
그래도       : 0.783
너무        : 0.773
그런데       : 0.773
조금        : 0.772
그렇게       : 0.768

[+] sg: 1, window: 15, negative: 0
천둥        : 0.452
빛살        : 0.446
편향성       : 0.423
김공        : 0.418
Fast      : 0.415

[+] sg: 1, window: 15, negative: 2
그래도       : 0.788
괜찮        : 0.772
그런다       : 0.74
너무        : 0.737
콧방귀       

In [27]:
for model in models:
    print(f"\n[+] sg: {model.sg}, window: {model.window}, negative: {model.negative}")
    for w in model.wv.most_similar(positive=["잘"], negative=[], topn=5):
        print(f"{w[0]:10}: {round(w[1], 3)}")
        


[+] sg: 0, window: 5, negative: 10
많이        : 0.775
널리        : 0.637
오래        : 0.617
자주        : 0.614
제대로       : 0.606

[+] sg: 0, window: 15, negative: 10
많이        : 0.743
오래        : 0.589
널리        : 0.586
자주        : 0.571
섬세        : 0.56

[+] sg: 0, window: 30, negative: 10
많이        : 0.703
뛰어나       : 0.586
자주        : 0.554
솜씨        : 0.55
똑똑        : 0.548

[+] sg: 1, window: 5, negative: 10
많이        : 0.714
제대로       : 0.692
휼륭한       : 0.687
좋         : 0.679
오래        : 0.673

[+] sg: 1, window: 15, negative: 10
좋         : 0.775
매우        : 0.771
많이        : 0.748
처럼        : 0.745
알려져       : 0.743

[+] sg: 1, window: 30, negative: 10
매우        : 0.864
알려져       : 0.841
처럼        : 0.805
좋         : 0.798
보다        : 0.797

[+] sg: 1, window: 15, negative: 0
右肩        : 0.482
가까운가      : 0.446
급부상       : 0.41
그대와       : 0.409
after     : 0.401

[+] sg: 1, window: 15, negative: 2
좋         : 0.794
매우        : 0.784
많이        : 0.761
처럼        : 0.744
어서       

In [29]:
for model in models:
    print(f"\n[+] sg: {model.sg}, window: {model.window}, negative: {model.negative}")
    for w in model.wv.most_similar(positive=["많이"], negative=[], topn=5):
        print(f"{w[0]:10}: {round(w[1], 3)}")
        


[+] sg: 0, window: 5, negative: 10
잘         : 0.775
자주        : 0.773
널리        : 0.715
큰         : 0.703
많         : 0.642

[+] sg: 0, window: 15, negative: 10
잘         : 0.743
자주        : 0.739
큰         : 0.683
널리        : 0.662
많         : 0.644

[+] sg: 0, window: 30, negative: 10
잘         : 0.703
자주        : 0.69
큰         : 0.645
많         : 0.643
흔한        : 0.623

[+] sg: 1, window: 5, negative: 10
자주        : 0.784
많         : 0.771
잘         : 0.714
비교적       : 0.702
읽혀진       : 0.699

[+] sg: 1, window: 15, negative: 10
많         : 0.783
매우        : 0.769
대부분       : 0.767
특히        : 0.749
잘         : 0.748

[+] sg: 1, window: 30, negative: 10
대부분       : 0.816
매우        : 0.811
비교        : 0.801
높         : 0.79
잘         : 0.783

[+] sg: 1, window: 15, negative: 0
SLOW      : 0.451
레오폴드      : 0.416
덴젤        : 0.413
Forgive   : 0.411
Sailfish  : 0.411

[+] sg: 1, window: 15, negative: 2
매우        : 0.796
대부분       : 0.784
많         : 0.768
잘         : 0.761
높       

In [30]:
for model in models:
    print(f"\n[+] sg: {model.sg}, window: {model.window}, negative: {model.negative}")
    for w in model.wv.most_similar(positive=["조금"], negative=[], topn=5):
        print(f"{w[0]:10}: {round(w[1], 3)}")
        


[+] sg: 0, window: 5, negative: 10
가끔        : 0.793
약간        : 0.786
이따금       : 0.747
반반        : 0.62
살짝        : 0.616

[+] sg: 0, window: 15, negative: 10
이따금       : 0.749
가끔        : 0.749
약간        : 0.698
반반        : 0.573
한참        : 0.546

[+] sg: 0, window: 30, negative: 10
약간        : 0.735
이따금       : 0.703
가끔        : 0.696
살짝        : 0.575
다소        : 0.519

[+] sg: 1, window: 5, negative: 10
약간        : 0.84
꽤         : 0.758
상당히       : 0.741
나아진다      : 0.739
달라져       : 0.738

[+] sg: 1, window: 15, negative: 10
약간        : 0.8
꽤         : 0.761
굉장히       : 0.753
상당히       : 0.748
다소        : 0.728

[+] sg: 1, window: 30, negative: 10
약간        : 0.811
너무        : 0.786
꽤         : 0.78
상당히       : 0.774
못         : 0.772

[+] sg: 1, window: 15, negative: 0
로프그렌      : 0.427
웅탄        : 0.419
차임        : 0.415
정신상       : 0.414
칼도        : 0.408

[+] sg: 1, window: 15, negative: 2
약간        : 0.797
너무        : 0.762
꽤         : 0.754
만큼        : 0.743
상당히       :

In [39]:
for i, model in enumerate(models):
    print(f"\n[{i}] sg: {model.sg}, window: {model.window}, negative: {model.negative}")
    for w in model.wv.most_similar(positive=["깜짝"], negative=[], topn=5):
        print(f"{w[0]:10}: {round(w[1], 3)}")
        


[0] sg: 0, window: 5, negative: 10
놀란다       : 0.557
무심코       : 0.551
뭔가        : 0.55
어쩐지       : 0.546
매번        : 0.545

[1] sg: 0, window: 15, negative: 10
놀란다       : 0.642
장난        : 0.626
뭔가        : 0.621
해한다       : 0.611
정말        : 0.605

[2] sg: 0, window: 30, negative: 10
해한다       : 0.623
정말        : 0.61
멋진        : 0.608
놀랐        : 0.608
뭔가        : 0.608

[3] sg: 1, window: 5, negative: 10
놀랬        : 0.796
터뜨린다      : 0.732
어리둥절      : 0.731
열받        : 0.731
어김없이      : 0.718

[4] sg: 1, window: 15, negative: 10
놀랬        : 0.794
긴가민가      : 0.779
울컥        : 0.773
놀랐        : 0.771
오랜만       : 0.76

[5] sg: 1, window: 30, negative: 10
오랜만       : 0.819
울컥        : 0.794
그리웠       : 0.787
정말        : 0.783
은데요       : 0.777

[6] sg: 1, window: 15, negative: 0
문무        : 0.421
소완규       : 0.411
배닝        : 0.405
포강        : 0.403
김석휘       : 0.399

[7] sg: 1, window: 15, negative: 2
글썽이       : 0.766
울먹이       : 0.762
시큰둥       : 0.754
오래간만      : 0.751
오랜만      